In [1]:
%matplotlib inline

In [4]:
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import sys
import numpy as np
import time

from math import sqrt

from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.ensemble import (GradientBoostingClassifier
                              , RandomForestClassifier
                              , AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression

sns.set_style('ticks')

In [1]:
from IPython.display import display, Markdown
display(Markdown("""
## The preamble handles a few imports.
## It also loads common functions and dicts: 

## Bosch challenge specific:
* **getMCC**(tp,tn,fp,fn)

## ML/Analytics functions:
* **compare_train_test**(clf, ds_train, label_train, ds_test, label_test, mva='MVA', bins=50, use_vote=None, log=False)
* **plot_classifier_output**( pred_train, pred_test, y_train, y_test, multipagepdf=None, bins = None, normalised = True )
* **plot_correlations**(data,label='', \*\*kwds)
* **optimisePars**(mva, points, data , classes, fraction=0.7, score = 'log_loss', cvs=5)

---

## Various
* **showUniques**(df)
* **ensure_dir**(directory)
* **printBumper**(text, c='=', n=-1)
* **intersec**(d1, d2)
* **union**(d1, d2)

---

## Color dictionaries:
* **Tableau10**
* **Tableau10_Light**
* **Tableau10_Medium**
* **Tableau_20**
* **ColorBlind10**
"""))


## The preamble handles a few imports.
## It also loads common functions and dicts: 

## Bosch challenge specific:
* **getMCC**(tp,tn,fp,fn)

## ML/Analytics functions:
* **compare_train_test**(clf, ds_train, label_train, ds_test, label_test, mva='MVA', bins=50, use_vote=None, log=False)
* **plot_classifier_output**( pred_train, pred_test, y_train, y_test, multipagepdf=None, bins = None, normalised = True )
* **plot_correlations**(data,label='', \*\*kwds)
* **optimisePars**(mva, points, data , classes, fraction=0.7, score = 'log_loss', cvs=5)

---

## Various
* **showUniques**(df)
* **ensure_dir**(directory)
* **printBumper**(text, c='=', n=-1)
* **intersec**(d1, d2)
* **union**(d1, d2)

---

## Color dictionaries:
* **Tableau10**
* **Tableau10_Light**
* **Tableau10_Medium**
* **Tableau_20**
* **ColorBlind10**


In [6]:
def getMCC(tp,tn,fp,fn):
    """
    Calculates Matthews correlation coefficient based on confusion matrix. 
    
    For model scoring the sklearn.metrics.matthews_corrcoef(y_true, y_pred) 
    can be used.
    
    Argument:
    tp,tn - Number of true positive and true negative observations
    fp,fn - Number of false positive and false negative observations
    """
    num = (tp*tn)-(fp*fn)
    den = sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    return float(num)/float(den)

In [20]:
# %load ../../pythonTools/plotting/plot.py
#Function to plot histogram with errorbars
def _hist_errorbars( data, xerrs=True, *args, **kwargs) :
  """
  Plot a histogram with error bars. 
  Accepts any kwarg accepted by either numpy.histogram or pyplot.errorbar
  """
  import numpy as np
  import matplotlib.pyplot as plt
  import inspect

  # pop off normed kwarg, since we want to handle it specially
  norm = False
  if 'normed' in list(kwargs.keys()) :
    norm = kwargs.pop('normed')

  # retrieve the kwargs for numpy.histogram
  histkwargs = {}
  for key, value in kwargs.items() :
    if key in inspect.getargspec(np.histogram).args :
      histkwargs[key] = value

  histvals, binedges = np.histogram( data, **histkwargs )
  yerrs = np.sqrt(histvals)

  if norm :
    nevents = float(sum(histvals))
    binwidth = (binedges[1]-binedges[0])
    histvals = histvals/nevents/binwidth
    yerrs = yerrs/nevents/binwidth

  bincenters = (binedges[1:]+binedges[:-1])/2

  if xerrs :
    xerrs = (binedges[1]-binedges[0])/2
  else :
    xerrs = None

  # retrieve the kwargs for errorbar
  ebkwargs = {}
  for key, value in kwargs.items() :
    if key in inspect.getargspec(plt.errorbar).args :
      ebkwargs[key] = value
    if key == 'color':
      ebkwargs['ecolor'] = value
      ebkwargs['mfc'] = value
    if key == 'label':
      ebkwargs['label'] = value

  out = plt.errorbar(bincenters, histvals, yerrs, xerrs, fmt="o", mec='black', ms=8, **ebkwargs)


  if 'log' in list(kwargs.keys()) :
    if kwargs['log'] :
      plt.yscale('log')

  if 'range' in list(kwargs.keys()) :
    plt.xlim(*kwargs['range'])

  return out

#Function to plot classifier output
def plot_classifier_output( pred_train, pred_test, y_train, y_test,
              multipagepdf=None, bins = None, normalised = True ):
  """
  Plots classifier output. 
  Works nicely with yandex/rep folding classifiers
  
  Arguments:
  pred_train: predictions for the trainings sample from the classifier.

  pred_test: predictions for the test sample from the classifier.

  y_train: true labels for the train sample. 

  y_test: true labels for the test sameple.

  multipagepdf: argument to pass multipagepdf instance.

  bins: binning for the resulting plot.

  normalised: control whether the plots are drawn normalised.
  """
  import numpy as np
  import matplotlib.pyplot as plt
  import inspect
  #Set binning
  binning = ( bins if bins is not None
              else np.linspace(np.min(pred_train), np.max(pred_train), 51) )

  #Set label to signal (=1) if label is None (for application )
  if y_train is None:
      y_train = np.ones(len(pred_train))
  if y_test is None:
      y_test = np.ones(len(pred_test))

  #Plot training sample
  plt.hist(pred_train[y_train<0.5], bins = binning, normed=normalised, histtype='stepfilled', color='b', alpha = 0.5,
           linewidth=0, label="Training Background")
  plt.hist(pred_train[y_train>0.5], bins = binning, normed=normalised, color='r', alpha = 0.5,
           linewidth=0, label="Training Signal", histtype='stepfilled')

  #Plot test sample
  _hist_errorbars( pred_test[y_test<0.5], xerrs=True, bins = binning, normed=normalised,
                  color='b', label="Test Background")
  _hist_errorbars( pred_test[y_test>0.5], xerrs=True, bins = binning, normed=normalised,
                  color='r', label="Test Signal")


  #plt.title("Classifier Output - Signal vs. Background and Test vs Training", fontsize=23)

  plt.xlabel("Classifier Output", fontsize=23)
  plt.ylabel("Normalised Events" if normalised else "Events", fontsize=23)

  #Restrict plot to binning area
  plt.xlim(binning[0], binning[-1])

  #Plot legend
  plt.legend(loc='best', fontsize=19)
  #Save plot
  #multipagepdf.savefig(bbox_inches = 'tight')
  #plt.close()
  #Plotted Classifier Output to {}".format(options.plots))
  return plt

def compare_train_test(clf, ds_train, label_train, 
                      ds_test, label_test, mva='MVA', 
                      bins=50, use_vote=None, log=False):

  """
  Plots output of the classifier for the training and test dataset.

  Arguments:
  clf: classifier object.

  ds_train: dataset used for training.
  
  label_train: labels for the training datasets.
  
  ds_test: dataset used to test the classifier performance.
  
  label_test: labels of the corresponding test dataset.
  
  mva: Name of the ML method (used as axis label, str).
  
  bins: Number of bins (int).

  use_vote: Function to be able to use vote functions for folding classifiers (CAUTION: is not strictly correct).

  log: In order to use a logarithmic scale on the plots (boolean).
  """
  import numpy as np
  import matplotlib.pyplot as plt
  import inspect
  decisions = []
  ns_train = len(ds_train[label_train>0.5])
  nb_train = len(ds_train[label_train<0.5])

  for X,y in ((ds_train, label_train), (ds_test, label_test)):
    if use_vote == None:
      d1 = clf.predict_proba(X[y>0.5])[:,1]#.ravel()
      d2 = clf.predict_proba(X[y<0.5])[:,1]#.ravel()
    else:
      d1 = clf.predict_proba(X[y>0.5],vote_function=use_vote)[:,1]#.ravel()
      d2 = clf.predict_proba(X[y<0.5],vote_function=use_vote)[:,1]#.ravel()

    decisions += [d1, d2]
      
  low = min(np.min(d) for d in decisions)
  high = max(np.max(d) for d in decisions)
  low_high = (low,high)
  _, ax = plt.subplots()
  ys, bins, _ = plt.hist(decisions[0],
           color='r', alpha=0.5, range=low_high, bins=bins,
           histtype='step', normed=True,
           label='S (train)',log=log)
  yb, _, _ = plt.hist(decisions[1],
           color='b', alpha=0.5, range=low_high, bins=bins,
           histtype='step', normed=True,
           label='B (train)',log=log)
  width = (bins[1] - bins[0])
  
  minY = 1./float(max(ns_train,nb_train))/width
  maxY = 6*max(yb.max(),ys.max())
  
  if log==True:
    plt.ylim([minY,maxY])
  hist, bins = np.histogram(decisions[2],
                            bins=bins, range=low_high, normed=True)
  
  scale = len(decisions[2]) / sum(hist)
  err = np.sqrt(hist * scale) / scale

  width = (bins[1] - bins[0])
  center = (bins[:-1] + bins[1:]) / 2

  fillx = [bins[0]]
  fillys = [ys[0]]
  fillyb = [yb[0]]
  for i,(x,y_s,y_b) in enumerate(zip(bins,ys,yb)):
    if i==0:
      continue
    if i==len(yb)-1:
      fillx+=[x,x,bins[i+1]]
      fillyb+=[yb[i-1],y_b,y_b]
      fillys+=[ys[i-1],y_s,y_s]
      continue
    fillx+=[x,x]
    fillys+=[ys[i-1],y_s]
    fillyb+=[yb[i-1],y_b]

  ax.fill_between(fillx,minY,fillys, color='r', alpha=0.5)
  ax.fill_between(fillx,minY,fillyb, color='b', alpha=0.5)
  
  plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='S (test)')

  hist, bins = np.histogram(decisions[3],
                            bins=bins, range=low_high, normed=True)
  scale = len(decisions[3]) / sum(hist)
  err = np.sqrt(hist * scale) / scale

  plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='B (test)')

  plt.xlabel(mva+" output")
  plt.ylabel("Arbitrary units")
  plt.legend(loc='best')
  return plt


# ========================================
# ========================================

def plot_correlations(data,label='', **kwds):
    """
    Calculate pairwise correlation between features.
    
    Arguments:
    data: Pandas.DataFrame on which the correlations are calculated.

    label: prefix for the plot title and file name.

    Extra arguments are passed on to DataFrame.corr()
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from pythonTools import ensure_dir
    # simply call df.corr() to get a table of
    # correlation values if you do not need
    # the fancy plotting
    corrmat = data.corr(**kwds)

    fig, ax1 = plt.subplots(ncols=1, figsize=(10,9))
    
    opts = {'cmap': plt.get_cmap("RdBu"),
            'vmin': -1, 'vmax': +1}
    heatmap1 = ax1.pcolor(corrmat, **opts)
    fig.colorbar(heatmap1, ax=ax1)


    ax1.set_title(label+" Correlations_{}vars".format(len(data.columns)-1))

    labels = corrmat.columns.values
    for ax in (ax1,):
        # shift location of ticks to center of the bins
        ax.set_xticks(np.arange(len(labels))+0.5, minor=False)
        ax.set_yticks(np.arange(len(labels))+0.5, minor=False)
        ax.set_xticklabels(labels, fontsize=12,minor=False, ha='right', rotation=70)
        ax.set_yticklabels(labels, fontsize=12,minor=False)
        
    fig.set_tight_layout(True)
    dir = 'plots'
    ensure_dir(dir)
    fig.savefig(dir+'/'+label+'_correlation_{}vars.pdf'.format(len(data.columns)))
    fig.savefig(dir+'/'+label+'_correlation_{}vars.png'.format(len(data.columns)))



In [23]:
# %load ../../pythonTools/utilities.py
'''
Module containing small general helper functions
'''

def showUniques(df):
  """
  Helper function that shows unique entries per DataFrame column.

  Arguments:
  df: the pandas DataFrame in question.
  """
  
  import pandas as pd
  from pandas import DataFrame 

  print("Number of rows: ", len(df))
  print("Number of unique values per column: ")
  for col in df.columns:
    print("Column {}: ".format(col), df[col].nunique())

#====================================
#====================================


def ensure_dir(directory):
  """When directory is not present, create it.

  Arguments: 
  directory: name of directory.
  """
  import os
  if not os.path.exists(directory):
    os.makedirs(directory)

#====================================
#====================================

def printBumper(text, c='=', n=-1):
  """Print a text with separating character. 

  Arguments: 
  text: text to be printed between separating character.

  c: separating character. 

  n: number of printed characters prior to the text (default -1: print as many characters as length of text)."""
  if n==-1:
    times = len(text)
  else:
    times = n
  print(c*times)
  print(text, ( (times-len(text)-1)*c ))
  print(c*times)

#====================================
#====================================

def intersec(d1, d2):
  """
  Returns list of intersecting elements.

  Arguments:
  d1/d2 - Objects that can be 'casted' as a sets.
  """
  return list(set(d1).intersection(set(d2)))

def union(d1,d2):
  """
  Returns list of union elements.

  Arguments:
  d1/d2 - Objects that can be 'casted' as a sets.
  """
  return list(set(d1).union(set(d2)))

In [26]:
# %load ../../pythonTools/ml_utilities.py
'''
Contains useful machine learning functions.
'''

def optimisePars(mva, points, data , classes, fraction=0.7, **kwargs):
    '''
    Funtion to optimise hyper-parameters. Follows sklearn example:
    "example-model-selection-grid-search-digits-py"
    
    Arguments:
    mva - multivariate method to optimise
    points - dictionary that holds optimisation points (hyper-parameters)
    data - your training data
    classes - true categories/classes/labels 
    fraction - fraction of training/test split
    **kwargs:
    scoring - score function to optimise the classifier (eg 'log_loss')
    cv - number of cross-validation folds or cross-validation generator
    n_jobs - number of parallel jobs for grid-search
    verbose - verbosity

    Returns:
    clf - GridSearchCV classifier.

    To-Do:
    - classification report does not exactly work every time. 
    '''
    import time
    # Tuning hyper-parameters for log_loss score
    
    # Splits data
    data_train, data_test, classes_train, classes_test =  train_test_split(
    data, classes, test_size=fraction, random_state=0)
    s =  time.time()
    clf = GridSearchCV(mva, points, cv=cvs,
                       scoring=score, n_jobs=njobs, 
                       verbose=vbs)
                       
    clf.fit(data_train, classes_train)

    print('GridSearch completed after ', (time.time()-s)/60.0, ' minutes.')
    print()
    print("Best parameters set found on training set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on training set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
      print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full training set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = classes_test, clf.predict_proba(data_test)
    print("Log loss score on test sample: ", log_loss(y_true, y_pred))
    
    return clf

In [27]:
# %load ../../pythonTools/plotting/nice_colors.py
'''
Color dictionaries from 
http://tableaufriction.blogspot.de/2012/11/finally-you-can-use-tableau-data-colors.html
'''

Tableau10 = { 
	'blue'   : (31/255.,119/255.,180/255.),
	'orange' : (255/255.,127/255.,14/255.),
	'green'  : (44/255.,160/255.,44/255.),
	'red'    : (214/255.,39/255.,40/255.),
	'purple' : (148/255.,103/255.,189/255.),
	'brown'  : (140/255.,86/255.,75/255.),
	'pink'   : (227/255.,119/255.,194/255.),
	'gray'   : (127/255.,127/255.,127/255.),
	'olive'  : (188/255.,189/255.,34/255.),
	'cyan'   : (23/255.,190/255.,207/255.)
}

Tableau10_Light	= {
	'blue'   : (174/255.,199/255.,232/255.),
	'orange' : (255/255.,187/255.,120/255.),
	'green'  : (152/255.,223/255.,138/255.),
	'red'    : (255/255.,152/255.,150/255.),
	'purple' : (197/255.,176/255.,213/255.),
	'brown'  : (196/255.,156/255.,148/255.),
	'pink'   : (247/255.,182/255.,210/255.),
	'gray'   : (199/255.,199/255.,199/255.),
	'olive'  : (219/255.,219/255.,141/255.),
	'cyan'   : (158/255.,218/255.,229/255.)
}

Tableau10_Medium = { 
	"blue"   : (114/255.,158/255.,206/255.),
	"orange" : (255/255.,158/255.,74/255.),
	"green"  : (103/255.,191/255.,92/255.),
	"red"    : (237/255.,102/255.,93/255.),
	"purple" : (173/255.,139/255.,201/255.),
	"brown"  : (168/255.,120/255.,110/255.),
	"pink"   : (237/255.,151/255.,202/255.),
	"gray"   : (162/255.,162/255.,162/255.),
	"olive"  : (205/255.,204/255.,93/255.),
	"cyan"   : (109/255.,204/255.,218/255.)
}

Tableau_20 = {
	"orange_light"    : (255/255.,187/255.,120/255.),
	"orange"          : (255./255.,127./255.,14/255.),
	"blue_light"      : (174./255.,199./255.,232/255.),
	"blue"            : (31./255.,119./255.,180/255.),
	"green_light"     : (152./255.,223./255.,138/255.),
	"green"           : (44./255.,160./255.,44/255.),
	"red_light"       : (255./255.,152./255.,150/255.),
	"red"             : (214./255.,39./255.,40/255.),
	"purple_light"    : (197./255.,176./255.,213/255.),
	"purple"          : (148./255.,103./255.,189/255.),
	"pink_light"      : (247./255,182./255.,210/255.),
	"pink"            : (227./255,119./255.,194/255.),
	"brown_light"     : (196./255,156./255.,148/255.),
	"brown"           : (140./255,86./255.,75/255.),
	"grey"            : (127./255,127./255.,127/255.),
	"grey_light"      : (199./255,199./255.,199/255.),
	"lime_light"      : (219./255,219./255.,141/255.),
	"lime"            : (188./255,189./255.,34/255.),
	"sapphire_light"  : (158./255,218./255.,229/255.),
	"sapphire"        : (23./255,190./255.,207/255.)
}

ColorBlind10 = {
	"azure"        : (0./255.,107./255.,164/255.),
	"orange"       : (255./255.,128./255.,14/255.),
	"grey_medium"  : (171./255.,171./255.,171/255.),
	"grey_dark"    : (89./255.,89./255.,89/255.),
	"blue_medium"  : (95./255.,158./255.,209/255.),
	"orange_dark"  : (200./255.,82./255.,0/255.),
	"grey"         : (137./255.,137./255.,137/255.),
	"blue_light"   : (162./255.,200./255.,236/255.),
	"orange_light" : (255./255.,188./255.,121/255.),
	"grey_light"   : (207./255.,207./255.,207/255.)

}